In [ ]:
!apt-get install libgeos-3.5.0
!apt-get install libgeos-dev
!pip install https://github.com/matplotlib/basemap/archive/master.zip
!pip install folium -U
!pip install branca

In [ ]:
from folium.plugins import HeatMapWithTime, MarkerCluster, Fullscreen
import folium
from folium.plugins import HeatMapWithTime
import pandas as pd
import sys


In [ ]:
# Load and process data
data = pd.read_csv('tows.csv')
data['Tow Date'] = pd.to_datetime(data['Tow Date'], errors='coerce')  # Handle invalid dates
data = data.dropna(subset=['Tow Date', 'lat', 'lon'])  # Remove rows with missing values

# Deafult Map only TimeSeries

In [ ]:

# Prepare data for HeatMapWithTime
time_series = sorted(data['Tow Date'].dt.strftime('%Y-%m-%d').unique())
heat_data = []

for date in time_series:
    daily_data = data[data['Tow Date'] == date]
    heat_data.append([[row['lat'], row['lon']] for index, row in daily_data.iterrows()])

# Check if heat_data is empty
if not any(heat_data):
    print("No valid data points found for heatmap.")
else:
    # Create map centered on the data's mean latitude and longitude
    map_center = [data['lat'].mean(), data['lon'].mean()]
    m = folium.Map(location=map_center, zoom_start=10)

    # Add HeatMapWithTime layer
    HeatMapWithTime(heat_data, index=time_series, radius=10).add_to(m)

    # Save to HTML
    m.save("tow_heatmap_with_time_Friday.html")
    print("Heatmap with time saved to 'tow_heatmap_with_time.html'.")


# Minimalist Map

In [ ]:

# Prepare data for HeatMapWithTime
time_series = sorted(data['Tow Date'].dt.strftime('%Y-%m-%d').unique())
heat_data = []

for date in time_series:
    daily_data = data[data['Tow Date'] == date]
    heat_data.append([[row['lat'], row['lon']] for index, row in daily_data.iterrows()])

# Check if heat_data is empty
if not any(heat_data):
    print("No valid data points found for heatmap.")
else:
    # Create map with a simpler, muted base style
    map_center = [data['lat'].mean(), data['lon'].mean()]
    m = folium.Map(location=map_center, zoom_start=10, tiles="CartoDB Positron")

    # Add HeatMapWithTime layer
    HeatMapWithTime(heat_data, index=time_series, radius=10).add_to(m)

    # Save to HTML
    m.save("tow_heatmap_with_time_simple_Friday.html")
    print("Heatmap with time saved to 'tow_heatmap_with_time_simple.html'.")


# Minimalist Map and Counts and Mark

In [ ]:

# Load and process data
data = pd.read_csv('tows.csv')

# Convert Tow Date to datetime and filter out invalid dates
data['Tow Date'] = pd.to_datetime(data['Tow Date'], errors='coerce')
data = data.dropna(subset=['Tow Date'])  # Drop rows with invalid dates

# Filter out rows with missing or invalid latitude/longitude values
data = data.dropna(subset=['lat', 'lon'])
data = data[(data['lat'] >= -90) & (data['lat'] <= 90) &
            (data['lon'] >= -180) & (data['lon'] <= 180)]

# Prepare data for HeatMapWithTime
time_series = sorted(data['Tow Date'].dt.strftime('%Y-%m-%d').unique())
heat_data = []

for date in time_series:
    daily_data = data[data['Tow Date'] == date]
    lat_lon_pairs = [[row['lat'], row['lon']] for index, row in daily_data.iterrows()]
    if lat_lon_pairs:  # Append only if there are valid points for this date
        heat_data.append(lat_lon_pairs)

# Check if heat_data is empty
if not heat_data:
    print("No valid data points found for heatmap.")
    sys.exit()

# Create a custom map with a nicer style
map_center = [data['lat'].mean(), data['lon'].mean()]
m = folium.Map(
    location=map_center,
    zoom_start=10,
    tiles="CartoDB Positron"  # Clean, modern map style
)

# Customize HeatMapWithTime
HeatMapWithTime(
    heat_data,
    index=time_series,
    radius=15,  # Adjust radius for a smoother appearance
    gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'yellow', 1.0: 'red'},  # Custom gradient
    auto_play=True,  # Automatically start the animation
    max_opacity=0.8,  # Make points more visible
    min_opacity=0.3  # Avoid very faint points
).add_to(m)

# Add a marker cluster for static points (optional)
marker_cluster = MarkerCluster().add_to(m)
for _, row in data.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=f"Tow Date: {row['Tow Date']}\nLocation: {row['Location']}",
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(marker_cluster)

# Add a full-screen control for better usability
Fullscreen(position='topright').add_to(m)

# Save to HTML
output_file = "tow_heatmap_with_Counts_V3_Friday.html"
m.save(output_file)
print(f"Enhanced heatmap with time saved to '{output_file}'.")


In [ ]:
!pip install openrouteservice
import openrouteservice
from openrouteservice import convert



In [ ]:
import openrouteservice
from openrouteservice import convert

client = openrouteservice.Client(key="YOUR_API_KEY")

coords = ((-74.0060, 40.7128), (-118.2437, 34.0522))  # New York City, Los Angeles

route = client.directions(coords)

distance = route['routes'][0]['summary']['distance'] / 1000  # Convert to km

print(distance)


In [ ]:
import openrouteservice
from openrouteservice import convert

# **Replace 'YOUR_API_KEY' with your actual OpenRouteService API key**
client = openrouteservice.Client(key="YOUR_ACTUAL_API_KEY")

coords = ((-74.0060, 40.7128), (-118.2437, 34.0522))

route = client.directions(coords)

distance = route['routes'][0]['summary']['distance'] / 1000

print(distance)